PROBLEMS:

Date and timestamps do not match for some analyses. RUN files for certain experiments (.D folders) show records of the same run ("002.D", "008.D", ..) repeated on separate days but documented in the same RUN file.
Possible that it indicates a fault run that was repeated the next day or days after.
Retain runtime as feature, any appearing as longer than a day (or other threshold, find) indicates an anomaly. Go back to actuals data to confirm. 



<h1>DeviceEngine Class

Dedicated engine for device data, inherited from Core Engine. Each DeviceEngine class object will represent a unique device with its own set of processing parameters and results.

In [1]:
from src.StreamPort.device.DeviceEngine import DeviceEngine
from src.StreamPort.core.ProcessingSettings import ProcessingSettings
from src.StreamPort.core.ProjectHeaders import ProjectHeaders

Create a new ProcessingSettings object 

In [2]:
settings = ProcessingSettings()

In [3]:
#specify path to get analyses from
base_dir = r'C:\Users\sande\Desktop\ExtractedSignals'

Creates an empty DeviceEngine object and prints it

In [4]:

dev = DeviceEngine(source = base_dir)
dev.print()


DeviceEngine 
  name: None 
  author: None 
  path: None 
  date: 2024-06-05 16:08:26.844526 
  analyses: 0 
  settings: 0 



In [5]:
dev1 = DeviceEngine()
dev1.print()
print(dev1._source)


DeviceEngine 
  name: None 
  author: None 
  path: None 
  date: 2024-06-05 16:08:26.855027 
  analyses: 0 
  settings: 0 

c:\Users\sande\Desktop\StreamPort


Add project headers. They can be passed as ProjectHeaders objects or dict

In [6]:
analist = dev1.find_analyses()

KeyboardInterrupt: 

In [ ]:
dev.add_headers(headers = {'name': 'Pressure Curve Analysis', 'author': 'Sandeep H.'})
dev.print()

Add processing settings

In [ ]:
dev.add_settings(settings)
dev.print()

<h1>DeviceAnalysis Class

Each DeviceAnalysis object is a child of the Analysis Class. It holds the details of an Analysis for each individual device.

In [ ]:
from src.StreamPort.device.DeviceAnalysis import DeviceAnalysis

#Creates an empty DeviceAnalysis object and prints it
devAnalysis = DeviceAnalysis()
devAnalysis.print()

 
DeviceEngine's find_analyses() method returns a DeviceAnalysis Object or a list of DeviceAnalysis objects, besides printing the dataframes for each unique Method, paired with the metadata(Date, Runtime) for each curve.

This method makes use of the source variable to accept a path to a directory containing analyses as an argument and find analyses from the target path.

The path can refer to a directory containing data for specific groups of experiments "210812_Gem 2021-08-12 09-49-10" or one such experiment containing its own set of method-related analysis data "210812_Gem--005.D", "210812_Gem--007.D", ..



Read analysis objects from engine.

In [ ]:
analyses = dev.find_analyses()

Each DeviceEngine object has an attribute _method_ids that records all methods encountered in the analysis of the current Device.

In [ ]:
print(dev._method_ids)

And an attribute _history to hold data on all experiments related to this device.

In [ ]:
print(dev._history)

In [ ]:
dev.print()

Add analyses objects that were found using find_analyses() to current device records.

In [ ]:
dev.add_analyses(analyses)

In [ ]:
for ana in dev._analyses:
    print("\n")
    ana.validate()
    print("Analysis Object : \n")
    print(f"Analysis : {ana.print()}")
    print("Data of Analysis : \n")
    print(ana.data)
    print("\n")

Add analyses in the form of individual DeviceAnalysis objects or a list of such objects.

In [ ]:
dev.print()

Remove analysis by name of DeviceAnalysis object, index in list of objects, or list of names or indexes

In [ ]:
dev.remove_analyses(0)
dev.print()

Retrieve DeviceAnalysis Objects or list of such objects by object name, index

In [ ]:
new_analyses = dev.get_analyses("Pac")

In [ ]:
for ana in new_analyses:
    ana.print()

In [ ]:
print(dev._analyses)

In [ ]:
for i in dev._analyses:
    i.print()

Plot analyses by calling inbuilt plot function and passing each object's index as argument

In [ ]:
for i in range(0,len(dev._analyses)):
    dev.plot_analyses(i)

Plot analyses by word or subword present in analysis name 

In [ ]:
dev.plot_analyses("Gem")

Plot analyses by word or subword present in analysis date

In [ ]:
dev.plot_analyses("2021-08")

Extract features for an analysis or group of analysis. This function, while calculating the features, also automatically adds these features DataFrames to the results dict attribute of the DeviceEngine object by calling the DeviceEngine object's inbuilt(inherited) add_results() function.

In [ ]:
print(dev._history)

In [ ]:
dev.get_features(2)

In [ ]:
dev.get_results(2)

In [ ]:
dev.remove_analyses(-1)

In [ ]:
dev.print()